In [2]:
# Installing required dependencies
%pip install torch torchvision scikit-learn matplotlib

  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB 435.7 kB/s eta 0:07:47
   ---------------------------------------- 0.0/203.1 MB 326.8 kB/s eta 0:10:22
   ---------------------------------------- 0.1/203.1 MB 393.8 kB/s eta 0:08:36
   ---------------------------------------- 0.1/203.1 MB 554.9 kB/s eta 0:06:06
   ---------------------------------------- 0.2/203.1 MB 700.2 kB/s eta 0:04:50
   ---------------------------------------- 0.2/203.1 MB 801.7 kB/s eta 0:04:14
   ---------------------------------------- 0.3/203.1 MB 842.9 kB/s eta 0:04:01
   ---------------------------------------- 0.3/203.1 MB 912.8 kB/s eta 0:03:43
   -----

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\untam\\anaconda3\\Lib\\site-packages\\torch\\lib\\torch_cpu.dll'
Consider using the `--user` option or check the permissions.



In [3]:
# Importing Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib.widgets import Button



ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# Transform for preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize between -1 and 1
])

# Download and load dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, persistent_workers=True)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Check data shapes
print(f"Train data: {len(train_loader.dataset)} samples")
print(f"Test data: {len(test_loader.dataset)} samples")


In [ ]:
#Defining the neural network
class DigitClassifier(nn.Module):
    def __init__(self):
        super(DigitClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)  # 10 classes for digits 0-9
        )
    
    def forward(self, x):
        return self.model(x)

# Initialize the model
model = DigitClassifier()
print(model)


In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")


In [ ]:
# Evaluation
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.numpy())

# Classification report
print(classification_report(y_true, y_pred, digits=4))


In [ ]:

# Create a blank canvas
canvas_size = 280
canvas = np.ones((canvas_size, canvas_size), dtype=np.float32) * 255  # White canvas


# Function to draw on canvas
def on_click(event):
    if event.xdata is not None and event.ydata is not None:  # If the click is within bounds
        x, y = int(event.xdata), int(event.ydata)
        cv2.circle(canvas, (x, y), 10, 0, -1)  # Draw a black circle on the canvas
        ax.imshow(canvas, cmap='gray')  # Update the image
        fig.canvas.draw()


# Function to classify the digit
def classify(event):
    global canvas
    digit = cv2.resize(canvas, (28, 28))  # Resize to 28x28
    digit = digit / 255.0  # Normalize to range [0, 1]
    digit = torch.tensor(digit, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

    model.eval()  # Put model in evaluation mode
    with torch.no_grad():
        output = model(digit)
        _, prediction = torch.max(output, 1)
    print(f"Predicted Digit: {prediction.item()}")


# Set up Matplotlib canvas
fig, ax = plt.subplots()
ax.imshow(canvas, cmap='gray')
ax.axis('off')  # Hide axes

# Add buttons for interaction
ax_classify = plt.axes([0.75, 0.01, 0.2, 0.05])  # Position for the button
btn_classify = Button(ax_classify, 'Classify')
btn_classify.on_clicked(classify)

fig.canvas.mpl_connect('motion_notify_event', on_click)  # Bind mouse motion to drawing
plt.show()
